<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/Freebase2Mtranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/castorini/BuboQA.git

Cloning into 'BuboQA'...
remote: Enumerating objects: 1383, done.
remote: Total 1383 (delta 0), reused 0 (delta 0), pack-reused 1383
Receiving objects: 100% (1383/1383), 361.07 KiB | 3.22 MiB/s, done.
Resolving deltas: 100% (908/908), done.


In [ ]:
!pip install fuzzywuzzy
%cd /content/BuboQA
!bash setup.sh 


/content/BuboQA
Running fetch_dataset script

--2021-05-18 15:45:16--  https://git.uwaterloo.ca/jimmylin/BuboQA-data/raw/master/SimpleQuestions_v2.tgz
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423435590 (404M) [application/x-gzip]
Saving to: ‘SimpleQuestions_v2.tgz’

SimpleQuestions_v2. 100%[===================>] 403.82M  97.0MB/s    in 4.3s    

2021-05-18 15:45:26 (93.9 MB/s) - ‘SimpleQuestions_v2.tgz’ saved [423435590/423435590]



Unzipping SimpleQuestions dataset...

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
./._SimpleQuestions_v2
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended head

In [ ]:
import pickle
with open('/content/BuboQA/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)

In [ ]:

from BuboQA.scripts.util import www2fb, clean_uri
i = 1
fout = open('translated.txt', 'w')
with open('/content/BuboQA/data/SimpleQuestions_v2/freebase-subsets/freebase-FB2M.txt', 'r') as fin:
  for line in fin:
    if i % 500000 == 0:
      print("line: {}".format(i))
    i+=1
    # if i>=300:
    #   break 
    parts = line.strip().split('\t')
    if len(parts) != 3:
      print("ERROR: line - {}".format(line))
      continue 
    subject = www2fb(parts[0])
    predicate = www2fb(parts[1])
    object = www2fb(parts[2])
    try:
      tmp_subject, tmp_object = subject, object
      subject, object = mid2name[subject], mid2name[object]
    except:
      continue
    for s in subject:
      for o in object:
        predicate = predicate.replace('.', '|||').replace('fb:', '')
        fout.write('{}|###|{}|###|{}|###|{}|###|{}\n'.format(tmp_subject, s, predicate, o, tmp_object))
    
fout.close()

    
    

line: 500000
line: 1000000
line: 1500000
line: 2000000
line: 2500000
line: 3000000
line: 3500000
line: 4000000
line: 4500000
line: 5000000
line: 5500000
line: 6000000
line: 6500000
line: 7000000
line: 7500000
line: 8000000
line: 8500000
line: 9000000
line: 9500000
line: 10000000
line: 10500000


In [ ]:
!cp translated.txt /content/drive/MyDrive/QA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
class ReverbKnowledgeBase:
  def __init__(self, path='translated.txt'):
    super().__init__()
    self.subjects = []
    self.predicates = []
    self.objects = []
    self.obj_mids = []
    i = 1
    with open(path, 'r') as fin:
      for line in fin:
        if i % 1000000 == 0:
          print("line: {}".format(i))
        i+=1
        tmp_subject, s, predicate, o, tmp_object = line.strip().split('|###|')
        self.subjects.append(s)
        self.predicates.append(predicate.split('|||')[-1])
        self.subjects.append(o)
        self.subjects.append(tmp_object)
        
		# df = pd.read_csv(path, sep='\t', header=None)
		# reverb_columns_name = ['rel']
		# df.columns = reverb_columns_name
		# df = df.dropna()
		# df = df.drop_duplicates()
		# self.KB = df
		# self.nodes = self.KB['arg1'].to_list()+self.KB['arg2'].to_list()
		# self.edges = self.KB['rel'].to_list()
    self.nodes_vectorizer = TfidfVectorizer()
    self.edges_vectorizer = TfidfVectorizer()
    self.subject_tfidf = self.nodes_vectorizer.fit_transform(self.subject)
    self.predicate_tfidf = self.edges_vectorizer.fit_transform(self.predicate)
		
  #   self.relations = {}
	# 	for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Indexing ...'):
	# 		if row['rel'] in self.relations:
	# 			self.relations[row['rel']].append((row['arg1'], index, row['conf']))
	# 			self.relations[row['rel']].append((row['arg2'], index, row['conf']))
	# 		else:
	# 			self.relations[row['rel']] = [(row['arg1'], index, row['conf'])]
	# 			self.relations[row['rel']].append((row['arg2'], index, row['conf']))
			
	# def tfidf_nodes_query(self, search_phrase, cutoff=50):
	# 	similarities = get_tf_idf_query_similarity(self.nodes_vectorizer, self.nodes_tfidf, search_phrase)
	# 	ranks = {k:v for k,v in zip(self.nodes, similarities)}
	# 	sorted_ranks = {k: v for k, v in sorted(ranks.items(), key=lambda item:item[1], reverse=True)[:min(len(ranks), cutoff)]}

	# 	return sorted_ranks

	# def tfidf_edges_query(self, search_phrase, cutoff=50):
	# 	similarities = get_tf_idf_query_similarity(self.edges_vectorizer, self.edges_tfidf, search_phrase)
	# 	ranks = {k:v for k,v in zip(self.edges, similarities)}
	# 	sorted_ranks = {k: v for k, v in sorted(ranks.items(), key=lambda item:item[1], reverse=True)[:min(len(ranks), cutoff)]}
	# 	return sorted_ranks

	# def tfidf_query(self, node='Bill Gates', edge='Born'):
	# 	nodes = self.tfidf_nodes_query(node)
	# 	edges = self.tfidf_edges_query(edge)
	# 	pruned = []
	# 	for node in nodes.keys():
	# 		for edge in edges.keys():
	# 			for item in self.relations[edge]:
	# 				if item[0]==node:
	# 					pruned.append((item[1], item[-1], nodes[node], edges[edge]))
	# 	sorted_pruned = sorted(pruned, key=lambda x:x[2]+x[3], reverse=True)
	# 	return sorted_pruned

In [ ]:
a = ReverbKnowledgeBase()

line: 1000000
line: 2000000
line: 3000000
line: 4000000
line: 5000000
line: 6000000
line: 7000000
line: 8000000
line: 9000000
line: 10000000
line: 11000000
line: 12000000
line: 13000000
line: 14000000
line: 15000000
line: 16000000
line: 17000000
line: 18000000
line: 19000000
line: 20000000
line: 21000000
line: 22000000
line: 23000000
line: 24000000
line: 25000000
line: 26000000
line: 27000000
line: 28000000
line: 29000000
line: 30000000
line: 31000000
line: 32000000
line: 33000000
line: 34000000
